In [1]:
import pandas as pd
from pathlib import Path
from pprint import pprint
from datasets import load_dataset

## To dataset format

In [2]:
from datasets import Dataset, DatasetDict
from src.utils import FormatPrompt, DialogueManager


def last_turn_rag_conv(data, FP, system_prompt):

    last_turn_dataset = []
    for (conver_id, pattern, sampling_id), conversation in data.groupby(["conver_id", "pattern", "sampling_stergy"]):
        conv_id = f"{conver_id}_{pattern}_{sampling_id}"

        converstaions = []
        conv = eval(conversation.to_json(orient="records", force_ascii=False,))
        
        DM = DialogueManager(system_prompt=system_prompt, log_path=".cache/chat.log")    
        for i, items in enumerate(conv):
            Q = items['questions']
            A = items['answers']
            F = items['retrieved_facts']
            C = items['citations']

            ## INPUT ##
            user, meta_user = FP.format_user(Q, facts=F) 
            assistant, meta_assistant = FP.format_assistant(A, C)
            
            ## Keep Conv
            turn = DM.data() + [user, assistant]
            converstaions.append(turn)
            
            ## Add history
            meta_user = {k:v for k,v in meta_user.items() if k in ["role", "content"]}
            meta_assistant = {k:v for k,v in meta_assistant.items() if k in ["role", "content"]}
            DM.add([meta_user, meta_assistant]) 
            DM.save()
        
        question, answer = converstaions[-1][:-1], converstaions[-1][-1]
        last_turn_dataset.append({"id": f"{conv_id}_turn_{i}", "question": question, "answer": answer,})
        
    return Dataset.from_list(last_turn_dataset)

## Pipeline

In [3]:
system_prompt = Path(f"src/prompt/context_with_citations.md").read_text().strip()
FP = FormatPrompt("contextcitations")

# Load data
dataset = load_dataset("weerayut/iq-rag")
test = dataset['test'].to_pandas()
train = dataset['train'].to_pandas()
validation = dataset['validation'].to_pandas()

# Proc
proc_test = last_turn_rag_conv(test, FP, system_prompt)
proc_train = last_turn_rag_conv(train, FP, system_prompt)
proc_validation = last_turn_rag_conv(validation, FP, system_prompt)

# Upload data
dataset = DatasetDict({"train": proc_train, "validation": proc_validation, "test": proc_test})
dataset.push_to_hub("weerayut/iq-rag-lastturn", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/706 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/weerayut/iq-rag-lastturn/commit/5089d524309b21e6fbfcfab4c09884ef6adfeddd', commit_message='Upload dataset', commit_description='', oid='5089d524309b21e6fbfcfab4c09884ef6adfeddd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/weerayut/iq-rag-lastturn', endpoint='https://huggingface.co', repo_type='dataset', repo_id='weerayut/iq-rag-lastturn'), pr_revision=None, pr_num=None)

## Check data

In [4]:
index = 3
question = proc_validation[index]['question']
answer = proc_validation[index]['answer']

proc_validation[index].keys()

dict_keys(['id', 'question', 'answer'])

In [5]:
pprint(question, sort_dicts=False, width=300)

[{'content': 'Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.', 'role': 'system'},
 {'content': 'ลูกเรียนไม่รู้เรื่องควรเริ่มประเมินจากอะไรดี?', 'role': 'user'},
 {'content': 'ควรเริ่มประเมินโดยพาเด็กไปพบแพทย์เฉพาะทางหรือจิตแพทย์เด็ก เพื่อรับการวินิจฉัยอย่างถูกต้อง ซึ่งจะประกอบด้วยการวัดเชาวน์ปัญญา การประเมินความสามารถทางการเรียน การตรวจร่างกาย การทดสอบทางจิตวิทยา และการรวบรวมข้อมูลจากครูผ่านแบบสอบถาม เพื่อหาสาเหตุและวางแผนการช่วยเหลืออย่างเหมาะสมต่อไป',
  'role': 'assistant'},
 {'content': 'การประเมินความสามารถทางการเรียนของเด็กควรใช้เครื่องมืออะไรบ้าง?', 'role': 'user'},
 {'content': 'การประเมินความสามารถทางการเรียนของเด็กควรใช้แบบทดสอบมาตรฐานที่ออกแบบมาเพื่อวัดทักษะทางวิชาการ เช่น การอ่าน การเขียน คิดเลข และความเข้าใจภาษา โดยประเมินร่วมกับการวัดเชาวน์ปัญญา การตรวจร่างกาย การทดสอบทางจิตวิทยา และข้อมูลจากครูผ่านแบบสอบถาม '
             'เพื่อให้ได้ภาพรวมที่ครบถ้วนและแม่นยำในการวินิจฉัยปั

In [6]:
pprint(answer, sort_dicts=False, width=300)

{'content': '{"answer": "ลูกที่มีปัญหาดิสกราเฟียควรได้รับการช่วยเหลืออย่างเหมาะสมโดยเริ่มจากการพาไปพบจิตแพทย์เด็กเพื่อรับการวินิจฉัยอย่างถูกต้อง พร้อมทั้งประเมินร่วมกับการวัดเชาวน์ปัญญา การตรวจร่างกาย การทดสอบทางจิตวิทยา และการรวบรวมข้อมูลจากครู เพื่อวางแผนการช่วยเหลือและรักษาอย่างเหมาะสมต่อไป", '
            '"citations": [2]}',
 'role': 'assistant'}


## Apply with chat template

In [7]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True,)

# Build prompt
text = tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True,)
print(text)

<|im_start|>system
Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.<|im_end|>
<|im_start|>user
ลูกเรียนไม่รู้เรื่องควรเริ่มประเมินจากอะไรดี?<|im_end|>
<|im_start|>assistant
ควรเริ่มประเมินโดยพาเด็กไปพบแพทย์เฉพาะทางหรือจิตแพทย์เด็ก เพื่อรับการวินิจฉัยอย่างถูกต้อง ซึ่งจะประกอบด้วยการวัดเชาวน์ปัญญา การประเมินความสามารถทางการเรียน การตรวจร่างกาย การทดสอบทางจิตวิทยา และการรวบรวมข้อมูลจากครูผ่านแบบสอบถาม เพื่อหาสาเหตุและวางแผนการช่วยเหลืออย่างเหมาะสมต่อไป<|im_end|>
<|im_start|>user
การประเมินความสามารถทางการเรียนของเด็กควรใช้เครื่องมืออะไรบ้าง?<|im_end|>
<|im_start|>assistant
การประเมินความสามารถทางการเรียนของเด็กควรใช้แบบทดสอบมาตรฐานที่ออกแบบมาเพื่อวัดทักษะทางวิชาการ เช่น การอ่าน การเขียน คิดเลข และความเข้าใจภาษา โดยประเมินร่วมกับการวัดเชาวน์ปัญญา การตรวจร่างกาย การทดสอบทางจิตวิทยา และข้อมูลจากครูผ่านแบบสอบถาม เพื่อให้ได้ภาพรวมที่ครบถ้วนและแม่นยำในการวินิจฉัยปัญหาการเรียนรู้<|im_end|>
<|im_star

## OPENAI

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4o",
    messages=question
)

print(response.choices[0].message.content)

{"answer":"เด็กที่มีปัญหาดิสกราเฟียควรได้รับการช่วยเหลือทางการเรียนโดยการจัดทำแผนการศึกษารายบุคคล (IEP) และได้รับการสอนเสริมจากผู้เชี่ยวชาญ รวมถึงใช้เทคโนโลยีช่วยเหลือ เช่น หนังสือเสียงหรือซอฟต์แวร์แปลงข้อความเป็นเสียง [1] ควรพาเด็กไปตรวจประเมินกับจิตแพทย์เด็กหรือจิตแพทย์เฉพาะทางด้าน LD เพื่อรับการวินิจฉัยและแนวทางการรักษาที่ถูกต้อง [2][4] การช่วยเหลือควรเริ่มตั้งแต่เนิ่น ๆ ด้วยความรัก ความเข้าใจ และกำลังใจจากครอบครัว [3] และควรมีความร่วมมือระหว่างผู้ปกครอง ครู และทีมแพทย์ เพื่อปรับวิธีการสอนให้เหมาะสมกับเด็ก [7]","citations":["1","2","3","4","7"]}


: 

: 